## Interacting with two and more tables in SQL

In [0]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect('chinook.db')

In [2]:
q = "SELECT * FROM sqlite_master WHERE type='table';"
pd.read_sql_query(q, conn)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


This is a fictitious DBMS that is similar to iTunes where customers can purchase songs. It has 11 tables. Each connected to the other by primary keys. 

In [3]:
q = "SELECT * FROM invoice_line limit 5"
pd.read_sql_query(q, conn)


,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1


In [4]:
q = "SELECT * FROM track limit 5"
pd.read_sql_query(q, conn)

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
q = "SELECT * FROM media_type limit 5"
pd.read_sql_query(q, conn)

,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [6]:
#Q Write a query that gathers data about the invoice with an invoice_id of 4.
q = "select t.track_id, t.name as track_name, mt.name as track_type, t.unit_price, il.quantity as quantity \
from invoice_line il INNER join track t on il.track_id=t.track_id \
INNER JOIN media_type mt on mt.media_type_id = t.media_type_id where invoice_id=4"
pd.read_sql_query(q, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


In [10]:
#Q Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased
q = "select al.title album, at.name artist, \
COUNT(*) tracks_purchased from invoice_line il \
INNER join track t on il.track_id=t.track_id INNER JOIN album al on al.album_id=t.album_id \
INNER JOIN artist at on at.artist_id=al.artist_id \
group by 1 ORDER by 3 desc limit 5"
pd.read_sql_query(q, conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


In [12]:
#Q Write a query that returns information about each employee and their supervisor. The report should include employees 
# even if they do not report to another employee and sorted alphabetically by the employee_name column.
q = "SELECT e1.first_name || ' ' || e1.last_name employee_name, e1.title employee_title, \
e2.first_name || ' ' || e2.last_name supervisor_name, e2.title supervisor_title \
from employee e1 LEFT join employee e2 on e1.reports_to=e2.employee_id ORDER by 1;"

pd.read_sql_query(q, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


In [13]:
#Q You have just returned from lunch to see another phone message on your desk: "Call Belle." Write a query that finds the contact details of a customer with a first_name containing Belle from the database.
q = "SELECT first_name, last_name, phone from customer where first_name LIKE '%Belle%';"
pd.read_sql_query(q, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


In [15]:
#Q Write a query that summarizes the purchases of each customer. 
# For the purposes of this exercise, we do not have any two customers with the same name.
q = "select c.first_name || ' ' || c.last_name customer_name, COUNT(invoice_id) number_of_purchases, \
SUM(i.total) total_spent, \
CASE WHEN sum(i.total)<40 THEN 'small spender' \
WHEN sum(i.total)>100 THEN 'big spender' \
else 'regular' \
END customer_category \
FROM invoice i  INNER JOIN customer c on i.customer_id=c.customer_id GROUP BY 1 ORDER by 1;"
pd.read_sql_query(q, conn)[:5]

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
